In [1]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown('# <span style="color:red">'+string+'</span>'))


if ('sc' in locals() or 'sc' in globals()):
    printmd('<<<<<!!!!! It seems that you are running in a IBM Watson Studio Apache Spark Notebook. Please run it in an IBM Watson Studio Default Runtime (without Apache Spark) !!!!!>>>>>')


In [ ]:
!pip install pyspark==2.4.5

In [2]:
try:
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
except ImportError as e:
    printmd('<<<<<!!!!! Please restart your kernel after installing Apache Spark !!!!!>>>>>')

In [3]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

we’ll work on clustering.

Let’s create our DataFrame again:


In [4]:
# delete files from previous runs
!rm -f hmp.parquet*

# download the file containing the data in PARQUET format
!wget https://github.com/IBM/coursera/raw/master/hmp.parquet
    
# create a dataframe out of it
df = spark.read.parquet('hmp.parquet')

# register a corresponding query table
df.createOrReplaceTempView('df')

--2020-05-01 22:22:27--  https://github.com/IBM/coursera/raw/master/hmp.parquet
Resolving github.com (github.com)... 140.82.118.4
Connecting to github.com (github.com)|140.82.118.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/IBM/skillsnetwork/raw/master/hmp.parquet [following]
--2020-05-01 22:22:27--  https://github.com/IBM/skillsnetwork/raw/master/hmp.parquet
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/skillsnetwork/master/hmp.parquet [following]
--2020-05-01 22:22:27--  https://raw.githubusercontent.com/IBM/skillsnetwork/master/hmp.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.16.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.16.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 932997 (911K) [application/octet-stream]
Savin

Let’s reuse our feature engineering pipeline.

In [12]:
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler, Normalizer, MinMaxScaler
from pyspark.ml.linalg import Vectors 
from pyspark.ml import Pipeline

indexer = StringIndexer(inputCol="class", outputCol="classIndex")
encoder = OneHotEncoderEstimator(inputCols=["classIndex"], outputCols=["categoryVec"])
vectorAssembler = VectorAssembler(inputCols=["x","y","z"],
                                  outputCol="features")

minmaxscaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")


pipeline = Pipeline(stages=[indexer, encoder, vectorAssembler, minmaxscaler])
model = pipeline.fit(df)
prediction = model.transform(df)
prediction.show()

+---+---+---+--------------------+-----------+----------+--------------+----------------+--------------------+
|  x|  y|  z|              source|      class|classIndex|   categoryVec|        features|      scaledFeatures|
+---+---+---+--------------------+-----------+----------+--------------+----------------+--------------------+
| 22| 49| 35|Accelerometer-201...|Brush_teeth|       6.0|(13,[6],[1.0])|[22.0,49.0,35.0]|[0.34920634920634...|
| 22| 49| 35|Accelerometer-201...|Brush_teeth|       6.0|(13,[6],[1.0])|[22.0,49.0,35.0]|[0.34920634920634...|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|       6.0|(13,[6],[1.0])|[22.0,52.0,35.0]|[0.34920634920634...|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|       6.0|(13,[6],[1.0])|[22.0,52.0,35.0]|[0.34920634920634...|
| 21| 52| 34|Accelerometer-201...|Brush_teeth|       6.0|(13,[6],[1.0])|[21.0,52.0,34.0]|[0.33333333333333...|
| 22| 51| 34|Accelerometer-201...|Brush_teeth|       6.0|(13,[6],[1.0])|[22.0,51.0,34.0]|[0.34920634920634...|
|

Now let’s create a new pipeline for kmeans.

In [16]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

kmeans = KMeans(featuresCol="features").setK(14).setSeed(1)
pipeline = Pipeline(stages=[vectorAssembler, kmeans])
model = pipeline.fit(df)
predictions = model.transform(df)

evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

Silhouette with squared euclidean distance = 0.41244594513295846


We have 14 different movement patterns in the dataset, so setting K of KMeans to 14 is a good idea. But please experiment with different values for K, do you find a sweet spot? The closer Silhouette gets to 1, the better.

https://en.wikipedia.org/wiki/Silhouette_(clustering)


In [20]:
# please change the pipeline the check performance for different K, feel free to use a loop
for i in range(2, 15):
    kmeans = KMeans(featuresCol="features").setK(i).setSeed(1)
    pipeline = Pipeline(stages=[vectorAssembler, kmeans])
    model = pipeline.fit(df)
    predictions = model.transform(df)

    evaluator = ClusteringEvaluator()

    silhouette = evaluator.evaluate(predictions)
    print("K = " + str(i) + ", Silhouette with squared euclidean distance = " + str(silhouette))

K = 2, Silhouette with squared euclidean distance = 0.6875664014387497
K = 3, Silhouette with squared euclidean distance = 0.6147915951361759
K = 4, Silhouette with squared euclidean distance = 0.6333227654128869
K = 5, Silhouette with squared euclidean distance = 0.5937447997439024
K = 6, Silhouette with squared euclidean distance = 0.592463658820136
K = 7, Silhouette with squared euclidean distance = 0.5484627422401509
K = 8, Silhouette with squared euclidean distance = 0.46686489256383346
K = 9, Silhouette with squared euclidean distance = 0.48034893889849645
K = 10, Silhouette with squared euclidean distance = 0.47370428136987536
K = 11, Silhouette with squared euclidean distance = 0.4819049717562352
K = 12, Silhouette with squared euclidean distance = 0.40964155503229643
K = 13, Silhouette with squared euclidean distance = 0.4153293521373778
K = 14, Silhouette with squared euclidean distance = 0.41244594513295846


We will choose K = 2

Now please extend the pipeline to work on the scaled features. You need to tell KMeans to use the scaled feature column and change the pipeline in order to contain the scaler stage as well.

In [22]:
kmeans = KMeans(featuresCol="scaledFeatures").setK(2).setSeed(1)
pipeline = Pipeline(stages=[vectorAssembler, minmaxscaler, kmeans])
model = pipeline.fit(df)

predictions = model.transform(df)

evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))


Silhouette with squared euclidean distance = 0.6875664014387497


Sometimes, inflating the dataset helps, here we multiply x by 10, let’s see if the performance inceases.

In [31]:
from pyspark.sql.functions import col
df_denormalized = df.select([col('*'),(col('x')*10)]).drop('x').withColumnRenamed('(x * 10)','x')

In [32]:
kmeans = KMeans(featuresCol="features").setK(2).setSeed(1)
pipeline = Pipeline(stages=[vectorAssembler, kmeans])
model = pipeline.fit(df_denormalized)
predictions = model.transform(df_denormalized)

evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

Silhouette with squared euclidean distance = 0.8263613179673074


Apache SparkML can be used to try many different algorithms and parametrizations using the same pipeline. Please change the code below to use GaussianMixture over KMeans. Please use the following link for your reference.

https://spark.apache.org/docs/latest/ml-clustering.html#gaussian-mixture-model-gmm


In [33]:
from pyspark.ml.clustering import GaussianMixture

gmm = GaussianMixture(featuresCol="scaledFeatures").setK(2).setSeed(1)
pipeline = Pipeline(stages=[vectorAssembler, minmaxscaler, gmm])

model = pipeline.fit(df)

predictions = model.transform(df)

evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

Silhouette with squared euclidean distance = 0.3521542042918571
